In [2]:
import datetime

import numpy as np
import pandas as pd
import pandas_datareader as pdr  # IF NECESSARY, from terminal: pip install pandas_datareader
from numpy.random import default_rng

pd.set_option("display.max_rows", 10)  # display option for pandas
# more here: https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

ModuleNotFoundError: No module named 'pandas_datareader'

## A quick hit of Numpy

In [1]:
# create a random vector 
# every run of this --> diff #s
# see 3.2.2.2 in the textbook for why, 
# and how to prevent

rg = default_rng()
myray = rg.standard_normal(5)
print("myray:", myray) 

NameError: name 'default_rng' is not defined

In [ ]:
# q1 - indexing, pick the odd elements
myray[0::2]
myray[-2]


Booleans arrays: Asking a logic test on an array, returns the array, where each element has been tested against that logic and converted to the boolean answer.

In [ ]:
# q2 - (a) boolean array + (b) "masking"

# (a)
myray > 0 # you can do "comparison operators" on np AND pandas arrays

# (b)
mask = myray > 0
myray[mask]


What you just learned about masking and filtering can be done with dataframes!

## The main event - Pandas

Vocab
- series
- index 
- dataframe
- columns and names
- rows and index 
- multiindex 

In [ ]:
start = datetime.datetime(2017, 1, 1) # you can specify start and end dates this way
end = datetime.datetime(2021, 1, 27)
macro_df = pdr.data.DataReader(['GDP','CPIAUCSL','UNRATE'], 'fred', start, end)


In [ ]:
# preview the dataframe
macro_df # type the name of the df

In [ ]:
# shape# ABCD!!!
macro_df.shape   # no paren - attribute 
# macro_df.shape[1]

In [ ]:
# variable types 
macro_df.info()

In [ ]:
# Summary Stats
macro_df.describe() 

In [ ]:
# look at top X rows
macro_df.head(8)

In [ ]:
# look at bottom X rows
macro_df.tail(12)

In [ ]:
# grab one variable
macro_df['UNRATE']

In [ ]:
# grab two (or more) variables
macro_df[ ['UNRATE','GDP']   ] 

In [ ]:
# see column names
macro_df.columns

In [ ]:
# change column names
macro_df.columns = ['GDP','CPI','UNRATE']
macro_df.columns

# if you are changing a few names in a large dataset >>> .rename()

In [ ]:
# see index
macro_df.index

In [ ]:
# reset_index() and set_index()
macro_df.reset_index() # turns index into var

macro_df.reset_index().set_index('DATE') # turn var(s) into index

In [ ]:
# grab some random rows
macro_df.sample(10)

In [ ]:
# grab some rows (by position)
macro_df[30:37]

In [ ]:
# grab some rows (by value)
mask = macro_df['UNRATE'] > 6
macro_df.loc[mask]

In [ ]:
# create a variable
# (2 ways)
macro_df['high'] = macro_df['UNRATE'] > 6
macro_df = macro_df.assign(low = macro_df['UNRATE'] < 4
macro_df

## EDA

Stop here. Back to the lecture. 

### Part 1

Q0: Do each of the [EDA golden rules for initial data exploration](https://ledatascifi.github.io/ledatascifi-2023/content/03/02e_eda_golden.html) and write down your observations.
- **Important: What is the "key" or "unit level" that observations in this database describes?** 
    - This is defined in 3.2.5 and discussed in [3.2.2.2 via example](https://ledatascifi.github.io/ledatascifi-2023/content/03/02b_pandasVocab.html#the-shape-of-data)
    - The "key" levels in databases we will look at are often increments of time in the data, the type of entity describe (e.g. firm, person, state, country, industry), and combinations of entity and time. 
    - E.g. "firm" level, "firm-year" level

Q1: What is the second series above?

Q2: What is the frequency of the series?

Q3: What is the average ANNUAL GDP, based on the data?

## Part 2

Q4: Download the annual *real* gdp from 1960 to 2018 from FRED and compute the average annual percent change

Q5: Compute the average gdp percent change within *each decade*


## Part 3

First, I'll load January data on unemployment, the Case-Shiller housing index, and median household income in three states (CA/MI/PA). 

In [ ]:
# LOAD DATA AND CONVERT TO ANNUAL

start = 1990 # pandas datareader can infer these are years
end = 2018
macro_data = pdr.data.DataReader(['CAUR','MIUR','PAUR', # unemployment 
                                  'LXXRSA','DEXRSA','WDXRSA', # case shiller index in LA, Detroit, DC (no PA  available!)
                                  'MEHOINUSCAA672N','MEHOINUSMIA672N','MEHOINUSPAA672N'], #  
                                 'fred', start, end)
macro_data = macro_data.resample('Y').first() # get's the first observation for each variable in a given year

# CLEAN UP THE FORMATING SOMEWHAT

macro_data.index = macro_data.index.year
print("\n\n DATA BEFORE FORMATTING: \n\n")
print(macro_data[:20]) # see how the data looks now? ugly variable names, but its an annual dataset at least
macro_data.columns=pd.MultiIndex.from_tuples([
    ('Unemployment','CA'),('Unemployment','MI'),('Unemployment','PA'),
    ('HouseIdx','CA'),('HouseIdx','MI'),('HouseIdx','PA'),
    ('MedIncome','CA'),('MedIncome','MI'),('MedIncome','PA')
    ])
print("\n\n DATA AFTER FORMATTING: \n\n")
print(macro_data[:20]) # this is a dataset that is "wide", and now 
                       # the column variable names have 2 levels - var name, 
                       # and unit/state that variable applies to


Q6: for each decade and state, report the average annual CHANGE (level, not percent) in unemployment

Q7: for each decade and state, report the average annual PERCENT CHANGE in house prices and household income